In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
######################################################################## 100.0%##O#- #                                                                       
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [1]:
import os

In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [6]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [5]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [11]:
from openai import OpenAI

client=OpenAI(
    base_url="http://localhost:11434/v1/",
    api_key="ollama",
)

In [12]:
client

In [21]:
llm('Write about cow')

' Cows are large domesticated mammals that have been bred and raised as livestple for their meat, milk, leather, and other by-products. They belong to the family Bovidae within the order Artiodactyla or even -omorpha. Their most prominent features include a sturdy body, four legs, horns (in males) that can grow quite large in some species like those from Africa’s Savannah regions known as buffaloes and their characteristic mooing sound used for communication within the herd and with humans.\n\nCows are ruminants which means they have a unique digestive system consisting of four compartments – The mouth, esophagus (also called throat), reticulum, omasum, abomasum - each serving specific functions in their complex feeding habits. Ruminants can consume cellulose-rich materials like grass which are usually indigestible for other animals due to the lack of necessary enzymes and microbial helpers present within a cow’s system.\n\nCows, particularly breeds such as Holstein Friesian dairy cows

In [22]:
print(_)

 Cows are large domesticated mammals that have been bred and raised as livestple for their meat, milk, leather, and other by-products. They belong to the family Bovidae within the order Artiodactyla or even -omorpha. Their most prominent features include a sturdy body, four legs, horns (in males) that can grow quite large in some species like those from Africa’s Savannah regions known as buffaloes and their characteristic mooing sound used for communication within the herd and with humans.

Cows are ruminants which means they have a unique digestive system consisting of four compartments – The mouth, esophagus (also called throat), reticulum, omasum, abomasum - each serving specific functions in their complex feeding habits. Ruminants can consume cellulose-rich materials like grass which are usually indigestible for other animals due to the lack of necessary enzymes and microbial helpers present within a cow’s system.

Cows, particularly breeds such as Holstein Friesian dairy cows have

### Dockerized Ollama


```bash 
docker run -it \
    -v ollama:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama

### Pull a model

```bash
docker exec -it ollama bash
ollama pull phi3